### Chord Extractor with librosa and chroma anlysis
Testing triad detection and then 7th detection in the section of the chord to define the tetrad quality

In [2]:
import os
import librosa
import librosa.display
import numpy as np 
import importlib
import formExtractor as fem
importlib.reload(fem)
import matplotlib.pyplot as plt

In [3]:
def load_audio_files(path):
    audio_files = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".mp3"):
                audio_files.append(os.path.join(root, file))
    return audio_files

In [4]:
path = '/home/laura/aimir/'
collection = 'suno' #lastfm, suno, udio
song_files = path + collection + '/audio'
files = load_audio_files(song_files)
print(len(files))

96365


In [5]:
myFiles = files[:20]

In [6]:
id = 5
song = myFiles[id]

#song = '/home/laura/aimir/boomy/audio/14065870.mp3'
id_file = song.split('/')[-1].split('.')[0]
song

'/home/laura/aimir/suno/audio/0002d6bf-36e5-4f04-a075-5476f06b2c49.mp3'

In [7]:
#Get the tonality of the song

y, sr = librosa.load(song)

# Compute the chroma features using CQT
chroma_cq = librosa.feature.chroma_cqt(y=y, sr=sr)

# Sum chroma features over time to emphasize prominent pitches
chroma_vector = np.sum(chroma_cq, axis=1)
chroma_vector /= np.linalg.norm(chroma_vector)

# Modified Krumhansl-Schmuckler key profiles
major_profile = np.array([6.35, 2.23, 3.48, 2.33, 4.38, 4.09,
                          2.52, 5.19, 2.39, 3.66, 2.29, 2.88])
minor_profile = np.array([6.33, 2.68, 3.52, 5.38, 2.60, 3.53,
                          2.54, 4.75, 3.98, 2.69, 3.34, 3.17])

# Optionally adjust profiles
# minor_profile[0] *= 1.2  # Emphasize tonic in minor

# Normalize the profiles
major_profile /= np.linalg.norm(major_profile)
minor_profile /= np.linalg.norm(minor_profile)

key_names = ['C', 'C#', 'D', 'D#', 'E', 'F',
             'F#', 'G', 'G#', 'A', 'A#', 'B']
correlations = []

for i in range(12):
    # Rotate the key profiles
    major_profile_rotated = np.roll(major_profile, i)
    minor_profile_rotated = np.roll(minor_profile, i)

    # Compute the correlation with adjusted weights
    major_corr = np.dot(chroma_vector, major_profile_rotated) * 0.9
    minor_corr = np.dot(chroma_vector, minor_profile_rotated) * 1.1  # Increase minor influence

    correlations.append({
        'key': key_names[i],
        'mode': 'major',
        'correlation': major_corr
    })
    correlations.append({
        'key': key_names[i],
        'mode': 'minor',
        'correlation': minor_corr
    })

# Find the best matching key
best_match = max(correlations, key=lambda x: x['correlation'])
tonality = f"{best_match['key']} {best_match['mode']}"
print(f"Tonality: {tonality}")

# Sort correlations in descending order
correlations_sorted = sorted(correlations, key=lambda x: x['correlation'], reverse=True)
print("Correlation scores for all keys:")
for corr in correlations_sorted:
    print(f"{corr['key']} {corr['mode']}: {corr['correlation']:.4f}")


Tonality: B minor
Correlation scores for all keys:
B minor: 1.0787
D minor: 1.0588
F# minor: 1.0576
E minor: 1.0407
G minor: 1.0352
A minor: 1.0330
C# minor: 1.0205
D# minor: 1.0182
G# minor: 1.0078
C minor: 1.0073
A# minor: 0.9966
F minor: 0.9843
D major: 0.8840
G major: 0.8639
A major: 0.8594
B major: 0.8326
E major: 0.8285
C major: 0.8252
F# major: 0.8227
F major: 0.8172
A# major: 0.8160
C# major: 0.8018
D# major: 0.7993
G# major: 0.7894


In [8]:
#correct sharp or flat tonality, by choosing the one with less alterations.


In [15]:
import triadExtractor as te
triads = te.TriadExtractor(hop_length=1024)
chords = triads.extract_chords(song, window_range=40, threshold=0.25, check_on_beat=True) #window_range minimum is 1
chords

[ChordChange(chord='Bmaj7', timestamp=np.float64(0.03931972789115645)),
 ChordChange(chord='Bm7', timestamp=np.float64(1.0145578231292516)),
 ChordChange(chord='D', timestamp=np.float64(1.966575963718821)),
 ChordChange(chord='A', timestamp=np.float64(2.941814058956916)),
 ChordChange(chord='B7', timestamp=np.float64(3.8938321995464853)),
 ChordChange(chord='Bm7', timestamp=np.float64(4.845850340136055)),
 ChordChange(chord='D', timestamp=np.float64(5.82108843537415)),
 ChordChange(chord='A', timestamp=np.float64(6.773106575963719)),
 ChordChange(chord='Bm7', timestamp=np.float64(7.725124716553288)),
 ChordChange(chord='G', timestamp=np.float64(8.677142857142858)),
 ChordChange(chord='D', timestamp=np.float64(9.652380952380954)),
 ChordChange(chord='A', timestamp=np.float64(10.581179138321996)),
 ChordChange(chord='G', timestamp=np.float64(11.556417233560092)),
 ChordChange(chord='Dm', timestamp=np.float64(13.483673469387755)),
 ChordChange(chord='A', timestamp=np.float64(14.4589115646